# To-do
- [ ] use snp_config.yml for covered call std
- [ ] rename std to sdev
- [ ] make raw option contracts for cc_long
- [ ] qualify the options and keep only one for each symbol (Call with lower strike)
- [ ] get the price, iv, hv and compute black_scholes price for the options
- [ ] make xPrice with get_prec based on greater of price and bs_price
- [ ] make cp_short, with the similar logic. Keep only one for each symbol (Put with higher strike)
- [ ] make protp_long and protc_short for protective calls and puts with similar logic
- [ ] make orders for all these covers and protects
- [ ] make orders for oprhans

In [10]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "SNP"

# Set the root
from from_root import from_root

ROOT = from_root()

import pandas as pd

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

import sys
from pathlib import Path

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util  # type: ignore

util.startLoop()

# Check account values

In [11]:
import asyncio

from src.ibfuncs import get_cushion, get_ib

with get_ib('SNP') as ib:
    acc = asyncio.run(get_cushion(ib))
display(acc)

{'cash': -410470.48,
 'cushion': 0.540426,
 'nlv': 605385.1,
 'init_margin': 305784.37,
 'equity_val': 604908.79,
 'maint_margin': 277977.53,
 'pnl_real': None,
 'pnl_unreal': 41886.3,
 'funds_avlbl': 299124.43}

# Get Base Data

In [42]:
# get portfolio, undsymbols and openorders

import numpy as np

from ibfuncs import df_chains, df_iv, get_ib, quick_pf, get_open_orders
from snp import get_snp_unds
from utils import how_many_days_old, pickle_me

unds_path = ROOT/'data'/'snp_unds.pkl'
chains_path = ROOT /'data'/'chains.pkl'

# age check
age_should_be_less_than = 0 # Note: unds_path is used to check the age for all files
recreate = False

if how_many_days_old(unds_path) is None or (how_many_days_old(unds_path) > age_should_be_less_than):
    recreate = True

if recreate:
    unds = get_snp_unds()
    pickle_me(unds, unds_path)

else:
    unds = pd.read_pickle(unds_path)

with get_ib('SNP') as ib:

    unds_iv=ib.run(df_iv(ib=ib, stocks=unds, msg='first run ivs'))

    no_price=unds_iv[unds_iv[['price', 'hv', 'iv']].isnull().any(axis=1)].symbol.to_list()
    second_unds_iv = ib.run(df_iv(ib=ib, stocks=no_price, sleep_time=10, msg='second run ivs'))

    pf_raw = quick_pf(ib)
    oo = get_open_orders(ib)

    # Set symbol as index for both dataframes
    cols = ['symbol', 'price', 'iv', 'hv']

    unds_iv = unds_iv[cols].set_index('symbol')
    second_unds_iv = second_unds_iv[cols].set_index('symbol')

    # Update unds_iv with non-null values from second_unds_iv 
    unds_iv.update(second_unds_iv)

    # unds_iv = unds_iv.set_index('symbol')[['hv', 'iv', 'price']]
    unds_iv.columns = ['und_' + col for col in unds_iv.columns]
    unds_iv = unds_iv.reset_index()

    # ... add und_price
    pf = pf_raw.merge(unds_iv, on='symbol', how='left')

    # Update und_price with mktPrice where und_price is NaN and mktPrice has a value
    pf.loc[pd.isna(pf['und_price']) & pd.notna(pf['mktPrice']), 'und_price'] = pf['mktPrice']

    # Merge the DataFrames on 'symbol'
    unds_iv = unds_iv.merge(pf[['symbol', 'und_price']], on='symbol', how='left')

    # Fill NaN values in 'und_price_x' with values from 'und_price_y'
    unds_iv['und_price'] = unds_iv['und_price_x'].fillna(unds_iv['und_price_y'])

    # Drop the unnecessary 'und_price_x' and 'und_price_y' columns
    unds_iv = unds_iv.drop(columns=['und_price_x', 'und_price_y'])

    # ...temp store the pf, oo
    pf_path = ROOT / 'data' / 'pf.pkl'
    oo_path = ROOT / 'data' / 'oo.pkl'

    if recreate:
        chains = asyncio.run(df_chains(ib, unds, msg='raw chains'))
    else:
        chains = pd.read_pickle(chains_path)

pickle_me(pf, pf_path)
pickle_me(oo, oo_path)
pickle_me(chains, chains_path)

raw chains: 100%|██████████| 237/237 [00:33<00:00,  7.05chunk/s]


# Classify positions
Positions are classified as follows:
- `cwp`: the perfect position that is protected and has a cover.
- `exposed`: stocks that need to be covered and protected.
- `uncovered`: stocks that need to be only covered by options.
- `unprotected`: stocks that need to be only protected by options.
- `orphaned`: options that have no underlying stocks positions.
- `covering`: options that are covering positions.
- `protecting`: options that are portecting positions.

In [43]:
# Sort by covered with protection pairs
right_order = {'C': 0, '0': 1, 'P': 2}

pf = pf.sort_values(
    by=['symbol', 'right'],
    key=lambda x: x.map(right_order) if x.name == 'right' else x
)

# Initialize strategy field with blank underscore
pf['strategy'] = 'tbd'

# Filter for options only
opt_pf = pf[pf.secType == 'OPT']

# Group by symbol and expiry to find matching calls and puts
straddled = (opt_pf.groupby(['symbol', 'expiry', 'strike'])
                      .filter(lambda x: (
                          # Must have exactly 2 rows (call and put)
                          len(x) == 2 and 
                          # Must have both C and P
                          set(x['right']) == {'C', 'P'} and
                          # Position signs must match
                          np.sign(x['position'].iloc[0]) == np.sign(x['position'].iloc[1])
                      )))

# Update strategy field for straddles
pf.loc[pf.index.isin(straddled.index), 'strategy'] = 'straddled'

# Filter for stocks and their associated options
cwp = (pf.groupby('symbol')
                      .filter(lambda x: (
                          # Must have exactly one STK row
                          (x.secType == 'STK').sum() == 1 and
                          # Must have 1 or 2 OPT rows
                          (x.secType == 'OPT').sum() in [1, 2]
                      )))

# Update strategy field for covered calls/puts
pf.loc[pf.index.isin(cwp[cwp.right == 'C'].index), 'strategy'] = 'covering'
pf.loc[pf.index.isin(cwp[cwp.right == 'P'].index), 'strategy'] = 'protecting'

# Update strategy field for stocks with both covering and protecting
stocks_with_both = pf[(pf.secType == 'STK') & 
                      pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                      pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
pf.loc[stocks_with_both.index, 'strategy'] = 'cwp'

# Update strategy field for stocks with covering but no protecting
stocks_covered_only = pf[(pf.secType == 'STK') &
                        pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                        ~pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
pf.loc[stocks_covered_only.index, 'strategy'] = 'unprotected'

# Update strategy field for stocks with protecting but no covering  
stocks_protected_only = pf[(pf.secType == 'STK') &
                          ~pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                          pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
pf.loc[stocks_protected_only.index, 'strategy'] = 'uncovered'


# Update strategy field for orphaned options
pf.loc[(pf.strategy == 'tbd') & (pf.secType == 'OPT'), 'strategy'] = 'orphaned'

# Update strategy field for exposed stock positions
pf.loc[(pf.strategy == 'tbd') & (pf.secType == 'STK'), 'strategy'] = 'exposed'

pickle_me(pf, pf_path)

In [44]:
# Check for null values in price, und_hv, and und_iv columns
null_rows = pf[pf[['und_price', 'und_hv', 'und_iv']].isnull().any(axis=1)]
print("Portfolio with null values in price, historical vol or implied vol:")
display(null_rows)

Portfolio with null values in price, historical vol or implied vol:


,secType,conId,symbol,expiry,strike,right,contract,position,mktPrice,mktVal,avgCost,unPnL,rePnL,und_price,und_iv,und_hv,strategy
49,OPT,729924008,IEFA,20241115,75.0,C,"Option(conId=729924008, symbol='IEFA', lastTra...",-4.0,0.20,-78.42,70.48,203.51,0.0,0.20,NaN,NaN,covering
48,STK,115826907,IEFA,,0.0,0,"Stock(conId=115826907, symbol='IEFA', right='0...",400.0,73.77,29508.00,66.12,3059.72,0.0,73.77,NaN,NaN,unprotected


# Cook orders 
<b>For existing positions</b>
- `Exposed` and `Uncovered` stocks should be covered
   - ...both for long (covered call) and short (covered put)
- `Exposed` and `Unprotected` stocks should be protected
   - ...both for long (protective put) and short (protective call)
- `Orphaned` stocks should be liquidated

<b>For rest of the symbols</b>
- Symbols with announcements in a week need to be straddled
- Remaining ones should have naked puts

In [45]:
# Get exposed and uncovered long
uncov = pf.strategy.isin(['exposed', 'uncovered'])
uncov_long = pf[uncov & (pf.position > 0)]

In [48]:
# Ready the chains for portfolio symbols
df_cc = (chains[chains.symbol.isin(uncov_long.symbol.unique())]
            .loc[(chains.dte.between(4, 8))]
            [['symbol', 'expiry', 'strike', 'dte']]
            .sort_values(['symbol', 'dte'])
            .reset_index(drop=True))

# Merge chains with underlying prices and volatilities
df_cc = df_cc.merge(unds_iv, on='symbol', how='left')

# Calculate standard deviation based on implied volatility and days to expiration
df_cc['std'] = df_cc.und_price * df_cc.und_iv * (df_cc.dte / 365)**0.5

# For each symbol and expiry, get 3 strikes above und_price + std

cc_std = 1
no_of_options = 2

cc_long = (
    df_cc.groupby(['symbol', 'expiry'])
    .apply(lambda x: x[x['strike'] > x['und_price'] + cc_std * x['std']]
                    .assign(diff=abs(x['strike'] - (x['und_price'] + cc_std * x['std'])))
                    .sort_values('diff')
                    .head(no_of_options), include_groups=False)
    .reset_index()
    .drop(columns=['level_2', 'diff'])
)


In [49]:
cc_long

,symbol,expiry,strike,dte,und_iv,und_hv,und_price,std
0,ABBV,20241115,207.5,5.15,0.19,0.20,200.51,4.49
1,ABBV,20241115,210.0,5.15,0.19,0.20,200.51,4.49
2,ACN,20241115,365.0,5.15,0.18,0.19,355.49,7.45
3,ACN,20241115,367.5,5.15,0.18,0.19,355.49,7.45
4,ADI,20241115,235.0,5.15,0.31,0.30,225.74,8.36
...,...,...,...,...,...,...,...,...
111,USB,20241115,52.5,5.15,0.22,0.30,50.24,1.33
112,WDC,20241115,72.5,5.15,0.33,0.41,69.64,2.70
113,WDC,20241115,73.0,5.15,0.33,0.41,69.64,2.70
114,WMT,20241115,88.0,5.15,0.27,0.14,84.84,2.76
